In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
print(os.listdir("../input"))

In [ ]:
import glob

from os.path import join as path_join
from scipy.stats import spearmanr, rankdata

In [ ]:
os.system('pip install ../input/sacremoses/sacremoses-master/ > /dev/null')
os.system('pip install ../input/transformers/transformers-master/ > /dev/null')

In [ ]:
package_path = '../input/radam-pytorch/RAdam' 
sys.path.append(package_path)

# quest-003-55-bert-07-3-valid

In [ ]:
folder = '../input/google-quest-challenge/'
pretrained_bert = "../input/pretrainedbertpytorch/pretrained-bert-pytorch/bert-base-uncased/"
model_weight_path1 = '../input/quest-003-55-bert-07-3-training-1/'
model_weight_path2 = '../input/quest-003-55-bert-07-3-training-2/'
model_weight_path3 = '../input/quest-003-55-bert-07-3-training-3/'

In [ ]:
import os
import re
import gc

import pickle  
import numpy as np
import pandas as pd
import random
import copy
import string
import time

import nltk
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from urllib.parse import urlparse
import math
from tqdm import tqdm

from spacy.lang.en import English
from scipy.stats import spearmanr

import warnings
warnings.simplefilter('ignore')

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torch.nn.utils.weight_norm import weight_norm

In [ ]:
import transformers
print("transformers:", transformers.__version__)

In [ ]:
SEED = 12345

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)

In [ ]:
import pickle

def read_pickle_from_file(pickle_file):
    with open(pickle_file,'rb') as f:
        x = pickle.load(f)
    return x

def write_pickle_to_file(pickle_file, x):
    with open(pickle_file, 'wb') as f:
        pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

## Load data

In [ ]:
train_df = pd.read_csv( f"{folder}train.csv" )
train_df.shape

In [ ]:
test_df = pd.read_csv( f"{folder}test.csv" )
test_df.shape

## Extract target variables

In [ ]:
target_table = [
    ['question_asker_intent_understanding',   'question'],
    ['question_body_critical',                'question'],
    ['question_conversational',               'question'],
    ['question_expect_short_answer',          'question'],
    ['question_fact_seeking',                 'question'],
    ['question_has_commonly_accepted_answer', 'question'],
    ['question_interestingness_others',       'question'],
    ['question_interestingness_self',         'question'],
    ['question_multi_intent',                 'question'],
    ['question_not_really_a_question',        'question'],
    ['question_opinion_seeking',              'question'],
    ['question_type_choice',                  'question'],
    ['question_type_compare',                 'question'],
    ['question_type_consequence',             'question'],
    ['question_type_definition',              'question'],
    ['question_type_entity',                  'question'],
    ['question_type_instructions',            'question'],
    ['question_type_procedure',               'question'],
    ['question_type_reason_explanation',      'question'],
    ['question_type_spelling',                'question'],
    ['question_well_written',                 'question'],
    ['answer_helpful',                        'answer'],
    ['answer_level_of_information',           'answer'],
    ['answer_plausible',                      'answer'],
    ['answer_relevance',                      'answer'],
    ['answer_satisfaction',                   'answer'],
    ['answer_type_instructions',              'answer'],
    ['answer_type_procedure',                 'answer'],
    ['answer_type_reason_explanation',        'answer'],
    ['answer_well_written',                   'answer'],    
]

input_columns = [
    'question_title', 
    'question_body',    
    'answer'
]

In [ ]:
target_question_columns = []
target_answer_columns = []

for table in target_table:
    if table[1] == 'question':
        target_question_columns.append( table[0] )
    elif table[1] == 'answer':
        target_answer_columns.append( table[0] )   
        
target_columns = target_question_columns + target_answer_columns

print( 'question:', len(target_question_columns) )
print( 'answer:', len(target_answer_columns) )
print( 'total:', len(target_columns) )

In [ ]:
for df in [train_df, test_df]:

    ## domain components
    df['domcom'] = df['url'].apply(lambda s: s.split('://')[1].split('/')[0].split('.'))

    # count components
    df['dom_cnt'] = df['domcom'].apply(lambda s: len(s))

    # extend length
    df['domcom'] = df['domcom'].apply(lambda s: s + ['none', 'none'])

    # components
    for ii in range(0,4):
        df['url_'+str(ii)] = df['domcom'].apply(lambda s: s[ii])

    # clean up
    df.drop('domcom', axis = 1, inplace = True)

### Features

In [ ]:
train_feature = pd.DataFrame()
test_feature = pd.DataFrame()

#### Text based features

In [ ]:
import nltk
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))

In [ ]:
for df, feature in [[train_df, train_feature], [test_df, test_feature]]:
    for column in input_columns:
        feature[column+'_total_length'] = df[column].apply(len)
        feature[column+'_capitals'] = df[column].apply(lambda comment: sum(1 for c in comment if c.isupper()))
        feature[column+'_caps_vs_length'] = feature.apply(lambda row: float(row[column+'_capitals'])/float(row[column+'_total_length']),axis=1)
        feature[column+'_num_exclamation_marks'] = df[column].apply(lambda comment: comment.count('!'))
        feature[column+'_num_question_marks'] = df[column].apply(lambda comment: comment.count('?'))
        feature[column+'_num_punctuation'] = df[column].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
        feature[column+'_num_symbols'] = df[column].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
        feature[column+"_num_chars"] = df[column].apply(lambda x: len(str(x)))
        feature[column+'_num_words'] = df[column].apply(lambda comment: len(comment.split()))
        feature[column+'_num_unique_words'] = df[column].apply(lambda comment: len(set(w for w in comment.split())))
        feature[column+'_words_vs_unique'] = feature[column+'_num_unique_words'] / feature[column+'_num_words']
        feature[column+'_num_smilies'] = df[column].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
        
        ## Number of stopwords in the text ##
        feature[column+"_num_stopwords"] = df[column].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
        
        ## Number of punctuations in the text ##
        feature[column+"_num_punctuations"] =df[column].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
        
        ## Number of title case words in the text ##
        feature[column+"_num_words_upper"] = df[column].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [ ]:
nlp = English()
sentencizer = nlp.create_pipe('sentencizer')
nlp.add_pipe(sentencizer)

ans_user_and_category=train_df[train_df[['answer_user_name', 'category']].duplicated()][['answer_user_name', 'category']].values
ans_user_and_category.shape

In [ ]:
def add_question_metadata_features(text):
    doc=nlp(text)
    indirect = 0
    choice_words=0
    reason_explanation_words = 0
    question_count = 0
    
    for sent in doc.sents:
        if '?' in sent.text and '?' == sent.text[-1]:
            question_count += 1
            for token in sent:
                if token.text.lower()=='why':
                    reason_explanation_words+=1
                elif token.text.lower()=='or':
                    choice_words+=1
    if question_count==0:
        indirect+=1
    
    return np.array([indirect, question_count, reason_explanation_words, choice_words])


def question_answer_author_same(df):
    q_username = df['question_user_name']
    a_username = df['answer_user_name']
    author_same=[]
    
    for i in range(len(df)):
        if q_username[i] == a_username[i]:
            author_same.append(int(1))
        else:
            author_same.append(int(0))
    return author_same


def add_external_features( df, feature ):
    feature['question_vs_answer_length'] = feature['question_body_num_words']/feature['answer_num_words']
    feature['q_a_author_same'] = question_answer_author_same(df)
    
    answer_user_cat = []
    for i in df[['answer_user_name', 'category']].values:
        if i in ans_user_and_category:
            answer_user_cat.append(int(1))
        else:
            answer_user_cat.append(int(0))
    feature['answer_user_cat'] = answer_user_cat
    
    handmade_features=[]
    for text in df['question_body'].values:
        handmade_features.append(add_question_metadata_features(text))

    feature = pd.concat( [ feature, pd.DataFrame( handmade_features, columns=['indirect', 'question_count', 'reason_explanation_words', 'choice_words'])], axis=1 )
    
    return feature

In [ ]:
train_feature = add_external_features( train_df, train_feature )
test_feature  = add_external_features( test_df, test_feature )

In [ ]:
for column in input_columns:
    print( "{} | Min: {}, Max: {}".format( column, train_feature[column+'_total_length'].min(), train_feature[column+'_total_length'].max() ) )
    
print( '=====' )

for column in input_columns:
    print( "{} | Min: {}, Max: {}".format( column, test_feature[column+'_total_length'].min(), test_feature[column+'_total_length'].max() ) )

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
symbol = ["'", '"', ':', ';', '.', ',', '-', '!', '?', "'s", ")", "(", "...", '``', "''", "/", "$", "%", "*", "&", "{", "}", "[", "]"]

def get_prevalent( texts, top_count=15 ):    
    tokenized_sents = [nltk.word_tokenize(i) for i in texts]
    tokenized_sents = [flatten for inner in tokenized_sents for flatten in inner]   
    
    #fdist = nltk.FreqDist(w for w in tokenized_sents if w not in stop_words + symbol)
    fdist = nltk.FreqDist(w.lower() for w in tokenized_sents if w.lower() not in stop_words + symbol)
    comments = fdist.most_common(top_count)
    
    return [word[0] for word in comments]

for column in input_columns:
    words = get_prevalent( train_df[column])
    print( column, words )
    
    for word in words:
        for df, feature in [[train_df, train_feature], [test_df, test_feature]]:
            feature[column+'_num'+word] = df[column].apply(lambda comment: comment.count(word))

#### count

In [ ]:
find = re.compile(r"^[^.]*")

train_df['netloc'] = train_df['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test_df['netloc'] = test_df['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

count_columns = ['question_title', 'question_user_name', 'answer_user_name', 'category', 'netloc']

for col in count_columns:
    value = train_df[col].value_counts()
    train_feature[col+'_count'] = train_df[col].map( value )
    test_feature[col+'_count']  = test_df[col].map( value ).fillna( 1 )

In [ ]:
for col in train_feature.columns: 
    train_mean = np.nanmean( train_feature[col].values )
    train_feature[col].fillna( train_mean, inplace=True )
    test_feature[col].fillna( train_mean, inplace=True ) 
    
print( "train: nan=", np.sum( np.sum( pd.isnull( train_feature ) ) ) )
print( "test : nan=", np.sum( np.sum( pd.isnull( test_feature  ) ) ) )

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_feature)

train_feature = pd.DataFrame(scaler.transform(train_feature), columns=train_feature.columns)
test_feature = pd.DataFrame(scaler.transform(test_feature), columns=test_feature.columns)

del scaler
gc.collect()

#### Label Preprocessing
- https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/100961

In [ ]:
def convert_label( df, result_df, column, count=10 ):
    labels = [ ( df[column].values>=(rate/count) ).astype(np.int) for rate in range( 1, count+1 ) ]

    columns = ['{}_{}'.format(column, i) for i in range(count)]
    labels = np.array( labels ).T
    
    label_df = pd.DataFrame( labels, columns=columns )
    result_df = pd.concat((result_df, label_df), axis=1)
    
    return result_df

In [ ]:
label_convert_count = 12
train_feature2 = pd.DataFrame()
test_feature2 = pd.DataFrame()

for column in train_feature.columns:
    train_feature2 = convert_label( train_feature, train_feature2, column, label_convert_count )
    test_feature2 = convert_label( test_feature, test_feature2, column, label_convert_count )   
    
train_feature = train_feature2.copy()
test_feature = test_feature2.copy()

del train_feature2, test_feature2

#### Encode

In [ ]:
features = ['question_title', 'question_user_name', 'answer_user_name']
limits = [6, 8, 8]

for col, limit in zip( features, limits ):
    value = train_df[col].value_counts()
    train_df['item_count'] = train_df[col].map( value )    
    train_df['item_value'] = train_df[col].copy()
    train_df.loc[train_df.item_count < limit, 'item_value'] = "___###___"
    
    test_df['item_count'] = test_df[col].map( value ).fillna( 1 )    
    test_df['item_value'] = test_df[col].copy()
    test_df.loc[test_df.item_count < limit, 'item_value'] = "___###___"    
    
    lb = LabelBinarizer()
    lb.fit( train_df['item_value'] )
    
    encode_train = lb.transform(train_df['item_value'])
    encode_test = lb.transform(test_df['item_value'])
    
    columns = ['LabelBinarizer_{}'.format(i) for i in range(encode_train.shape[1])]
    print( "{}: {}". format( col, len( train_df['item_value'].value_counts() ) ) )
    
    encode_train = pd.DataFrame( encode_train, columns=columns )
    train_feature = pd.concat((train_feature, encode_train), axis=1)

    encode_test = pd.DataFrame( encode_test, columns=columns )
    test_feature = pd.concat((test_feature, encode_test), axis=1)      
    
    del lb
    
train_df.drop( ['item_count', 'item_count'], axis=1, inplace=True )
test_df.drop( ['item_count', 'item_count'], axis=1, inplace=True )

In [ ]:
features = ['url_0', 'category']
enc = OneHotEncoder( handle_unknown='ignore' )
enc.fit( train_df[features] )

encode_train = enc.transform(train_df[features]).toarray()
encode_test = enc.transform(test_df[features]).toarray()

columns = ['encode_{}'.format(i) for i in range(encode_train.shape[1])]

encode_train = pd.DataFrame( encode_train, columns=columns )
train_feature = pd.concat((train_feature, encode_train), axis=1)

encode_test = pd.DataFrame( encode_test, columns=columns )
test_feature = pd.concat((test_feature, encode_test), axis=1)   

del encode_train, encode_test, enc

### remove nan

In [ ]:
for col in train_feature.columns: 
    train_mean = np.nanmean( train_feature[col].values )
    train_feature[col].fillna( train_mean, inplace=True )
    test_feature[col].fillna( train_mean, inplace=True ) 
    
print( "train: nan=", np.sum( np.sum( pd.isnull( train_feature ) ) ) )
print( "test : nan=", np.sum( np.sum( pd.isnull( test_feature  ) ) ) )

#### Text cleaning

In [ ]:
# https://www.kaggle.com/chenshengabc/from-quest-encoding-ensemble-a-little-bit-differen

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
#         df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
        df[col] = df[col].apply(lambda x: handle_contractions(x))  
        df[col] = df[col].apply(lambda x: fix_quote(x))   
    
    return df


train_df = clean_data( train_df, input_columns )
test_df = clean_data( test_df, input_columns )

del tokenizer

#### Bert

In [ ]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

class NeuralNet(nn.Module):
    def __init__(self, num_features, num_labels, pretrained_bert ):
        super(NeuralNet, self).__init__()
        
        self.bert = transformers.BertModel.from_pretrained( pretrained_bert )
        self.num_features = num_features
        self.num_labels = num_labels
        
        self.encoded_dropout = SpatialDropout( 0.2 )
        self.pooled_dropout = nn.Dropout( 0.2 )        
        
        self.feature_linear = nn.Sequential(
            nn.Linear( self.num_features, self.num_features ),
            nn.ReLU( inplace=True ),
            nn.Dropout( 0.2 ),            
        )
        
        dense_hidden_units = self.bert.config.hidden_size * 3 + self.num_features
        
        self.linear1 = nn.Linear( dense_hidden_units, dense_hidden_units )
        self.linear2 = nn.Linear( dense_hidden_units, dense_hidden_units )
        
        self.classifier = nn.Sequential(
            nn.Linear( dense_hidden_units, num_labels ),
        )
        
    def forward( self, ids, masks, segments, feature ):  
        
        feature_output = self.feature_linear( feature )
        
        sequence_output, pooled_output = self.bert( input_ids=ids, attention_mask=masks, token_type_ids=segments )
        sequence_output = self.encoded_dropout(sequence_output)
        pooled_output = self.pooled_dropout(pooled_output)        
        
        avg_pool = torch.mean( sequence_output, 1 )
        max_pool, _ = torch.max( sequence_output, 1 )
        
        h_conc = torch.cat( ( avg_pool, max_pool, pooled_output, feature_output ), 1 )
        
        h_conc_linear  = F.relu(self.linear1(h_conc))
        hidden = h_conc + h_conc_linear 
            
        h_conc_linear  = F.relu(self.linear2(hidden))
        hidden = hidden + h_conc_linear      
        
        return self.classifier( hidden )

In [ ]:
def compute_input_title_question( df, tokenizer, max_sequence_length=512 ):
    
    input_ids, input_masks, input_segments = [], [], []
    
    for _, instance in df.iterrows():
         
        title    = tokenizer.tokenize(instance.question_title)
        question = tokenizer.tokenize(instance.question_body)

        if (len(title)+len(question)+3) > max_sequence_length:
            if len(title) > 30:
                title = title[:30]
                
            question_len = max_sequence_length - len(title) - 3
            question = question[:question_len]
        
        #token = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] 
        #token_ids = tokenizer.convert_tokens_to_ids(token)
        
        title_ids    = tokenizer.convert_tokens_to_ids(title)
        question_ids = tokenizer.convert_tokens_to_ids(question)
        cls_ids      = tokenizer.convert_tokens_to_ids( ["[CLS]"] )   
        sep_ids      = tokenizer.convert_tokens_to_ids( ["[SEP]"] ) 
        
        token_ids = cls_ids + title_ids + sep_ids + question_ids + sep_ids
        padding = [0] * (max_sequence_length - len(token_ids))
        
        ids      = token_ids + padding
        masks    = [1]*len(token_ids) + padding
        segments = [0]*(len(title_ids)+2) + [1]*(len(question_ids)+1) + padding
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),   
    ]

In [ ]:
def compute_input_arrays( df, tokenizer, max_sequence_length=512 ):
    
    input_ids, input_masks, input_segments = [], [], []    
    t_max_len = 35     
    
    for _, instance in df.iterrows():
        
        title    = tokenizer.tokenize(instance.question_title)
        question = tokenizer.tokenize(instance.question_body)
        answer   = tokenizer.tokenize(instance.answer)

        if (len(title)+len(question)+len(answer)+4) > max_sequence_length:
            if len(title) > t_max_len:
                title = title[:t_max_len]
                
            question_len = ( max_sequence_length - len(title) - 4 ) // 2
            question = question[:question_len]
            
            answer_len = max_sequence_length - len(title) - len(question) - 4
            answer = answer[:answer_len]
        
        #token = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
        #token_ids = tokenizer.convert_tokens_to_ids(token)
        
        title_ids    = tokenizer.convert_tokens_to_ids(title)
        question_ids = tokenizer.convert_tokens_to_ids(question)
        answer_ids   = tokenizer.convert_tokens_to_ids(answer) 
        cls_ids      = tokenizer.convert_tokens_to_ids( ["[CLS]"] )   
        sep_ids      = tokenizer.convert_tokens_to_ids( ["[SEP]"] ) 
        
        token_ids = cls_ids + title_ids + sep_ids + question_ids + sep_ids + answer_ids + sep_ids
        padding = [0] * (max_sequence_length - len(token_ids))
        
        ids      = token_ids + padding
        masks    = [1]*len(token_ids) + padding
        segments = [0]*(len(title_ids)+len(question_ids)+3) + [1]*(len(answer_ids)+1) + padding
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),   
    ]

In [ ]:
n_splits = 6

'''
x_train = train_feature.values
y_train = train_df[target_columns].values

cv = KFold( n_splits=n_splits, random_state=SEED )
kfold_split = list( cv.split( x_train, y_train ) )

write_pickle_to_file( 'kfold_split_index.pkl', kfold_split )
'''

kfold_split = read_pickle_from_file( model_weight_path1 + 'kfold_split_index.pkl' )

In [ ]:
test_pred_datas = {}
test_pred_weights = {}

for column in target_columns:
    test_pred_datas[column] = np.zeros( len(test_df) )
    test_pred_weights[column] = 0.0
    
def add_test_pred_data( prediction, columns, weight ):
    for column_idx, column in enumerate( columns ):
        test_pred_datas[column] += weight * prediction[:, column_idx]  
        test_pred_weights[column] += weight    

In [ ]:
validation_datas = {}
validation_counts = {}

for column in target_columns:
    validation_datas[column] = np.zeros( len(train_df) )
    validation_counts[column] = np.zeros( len(train_df) )

def add_validation_data( prediction, columns, idx ):
    for column_idx, column in enumerate( columns ):
        validation_datas[column][idx] += prediction[:, column_idx]  
        validation_counts[column][idx] += 1.0    

In [ ]:
def mean_spearmanr_correlation_score( y_true, y_pred ):
    num_labels = y_pred.shape[1]
    score = np.nanmean( [ spearmanr( y_pred[:, idx], y_true[:, idx] ).correlation for idx in range(num_labels) ] )
    return score 

In [ ]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, features, labels = None):
        
        self.inputs   = inputs
        self.features = features
        
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        input_features  = self.features[idx]
        
        if self.labels is not None: # targets
            input_labels = self.labels[idx]
            return input_ids, input_masks, input_segments, input_features, input_labels
        
        return input_ids, input_masks, input_segments, input_features

    def __len__(self):
        return len(self.inputs[0])

In [ ]:
def model_test_validation( label_columns, train_inputs, test_inputs, x_train, x_test, weight_files ):
    
    if len(kfold_split) != len(weight_files):
        return
    
    batch_size   = 6    
    num_features = x_test.shape[1]
    num_labels   = len(label_columns)

    test_dataset = QuestDataset( test_inputs, x_test, None )
    test_loader = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size, shuffle=False )   
    
    for k, (train_idx, valid_idx) in enumerate(kfold_split):
        
        fname = weight_files[k]
        print( k+1, fname )
        
        x_train_valid = x_train[valid_idx]
        train_inputs_valid = [x[valid_idx] for x in train_inputs]

        valid_dataset = QuestDataset( train_inputs_valid, x_train_valid, None )
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False )
        
        model = NeuralNet( num_features, num_labels, pretrained_bert )
        model.to('cuda:0')

        model.load_state_dict( torch.load( fname ) )      
        model.eval()
     
        #====================
        #validation
        valid_preds_fold  = []

        with torch.no_grad():
            for ids, masks, segments, features in valid_loader:
                ids      = ids.cuda()
                masks    = masks.cuda()
                segments = segments.cuda()
                features = torch.tensor(features,dtype=torch.float32).cuda()
                y_pred = model( ids, masks, segments, features )
                valid_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

            valid_preds_fold = np.array( valid_preds_fold )
            add_validation_data( valid_preds_fold, label_columns, valid_idx )
            
        #====================
        #test
        test_preds_fold  = []

        with torch.no_grad():
            for ids, masks, segments, features in test_loader:
                ids      = ids.cuda()
                masks    = masks.cuda()
                segments = segments.cuda()
                features = torch.tensor(features,dtype=torch.float32).cuda()
                y_pred = model( ids, masks, segments, features )
                test_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

            test_preds_fold = np.array( test_preds_fold )
            add_test_pred_data( test_preds_fold, label_columns, 1.0 )            

        del model, valid_dataset, valid_loader
        torch.cuda.empty_cache()
        gc.collect()
        
    del test_dataset, test_loader

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained( pretrained_bert )

### question

In [ ]:
train_inputs = compute_input_title_question( train_df, tokenizer, max_sequence_length=512 )
test_inputs = compute_input_title_question( test_df, tokenizer, max_sequence_length=512 )

x_train = train_feature.values
x_test  = test_feature.values

print( x_train.shape )
print( x_test.shape )
print( len( target_question_columns ) )

weight_files = [
    model_weight_path1 + 'best_weights_003_55_7_3_question_1.pth',
    model_weight_path1 + 'best_weights_003_55_7_3_question_2.pth',
    model_weight_path2 + 'best_weights_003_55_7_3_question_3.pth',
    model_weight_path2 + 'best_weights_003_55_7_3_question_4.pth',
    model_weight_path3 + 'best_weights_003_55_7_3_question_5.pth',
    model_weight_path3 + 'best_weights_003_55_7_3_question_6.pth',
]

model_test_validation( target_question_columns, train_inputs, test_inputs, x_train, x_test, weight_files )

del x_train, x_test
del train_inputs, test_inputs

### answer

In [ ]:
train_inputs = compute_input_arrays( train_df, tokenizer, max_sequence_length=512 )
test_inputs = compute_input_arrays( test_df, tokenizer, max_sequence_length=512 )

x_train = train_feature.values
x_test  = test_feature.values

print( x_train.shape )
print( x_test.shape )
print( len( target_answer_columns ) )

weight_files = [
    model_weight_path1 + 'best_weights_003_55_7_3_answer_1.pth',
    model_weight_path1 + 'best_weights_003_55_7_3_answer_2.pth',
    model_weight_path2 + 'best_weights_003_55_7_3_answer_3.pth',
    model_weight_path2 + 'best_weights_003_55_7_3_answer_4.pth',
    model_weight_path3 + 'best_weights_003_55_7_3_answer_5.pth',
    model_weight_path3 + 'best_weights_003_55_7_3_answer_6.pth',
]

model_test_validation( target_answer_columns, train_inputs, test_inputs, x_train, x_test, weight_files )

del x_train, x_test
del train_inputs, test_inputs


In [ ]:
del train_feature, test_feature
del target_question_columns, target_answer_columns 

### validation

In [ ]:
validationT = pd.DataFrame()

for column in target_columns:
    preds = validation_datas[column]
    count = validation_counts[column]
    count = np.where( count < 0.5, 1.0, count )
    
    validationT[column] = preds / count

In [ ]:
mean_spearmanr_correlation_score( validationT.values, train_df[target_columns].values )

In [ ]:
validationT.head()

### test

In [ ]:
test_pred_weights

In [ ]:
test_predsT = pd.read_csv( f"{folder}sample_submission.csv" )

for column in target_columns:
    preds = test_pred_datas[column]
    weight = test_pred_weights[column]
    
    test_predsT[column] = preds / weight

In [ ]:
test_predsT.head()

In [ ]:
del target_columns, input_columns
del test_pred_datas, test_pred_weights
del validation_datas, validation_counts

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
blabla

# 006-55-gpt2-01-1

In [ ]:
!ls ../input/photostage1/

In [ ]:
folder = '../input/google-quest-challenge/'
pretrained_bert = "../input/pretrainedbertpytorch/pretrained-bert-pytorch/gpt2/"
model_weight_path1 = '../input/photostage1/best_weights_006-55-gpt2-01_question_1.pth'
model_weight_path2 = '../input/photostage2/best_weights_006-55-gpt2-01_question_2.pth'
model_weight_path3 = '../input/photostage3/best_weights_006-55-gpt2-01_question_3.pth'
model_weight_path4 = '../input/photostage4/best_weights_006-55-gpt2-01_question_4.pth'
model_weight_path5 = '../input/photostage5/best_weights_006-55-gpt2-01_question_5.pth'
model_weight_path6 = '../input/photostage6/best_weights_006-55-gpt2-01_question_6.pth'
model_weight_path7 = '../input/photostagea1/best_weights_006-55-gpt2-01_answer_1.pth'
model_weight_path8 = '../input/photostagea2/best_weights_006-55-gpt2-01_answer_2.pth'
model_weight_path9 = '../input/photostagea3/best_weights_006-55-gpt2-01_answer_3.pth'
model_weight_path10 = '../input/photostagea4/best_weights_006-55-gpt2-01_answer_4.pth'
model_weight_path11 = '../input/photostagea5/best_weights_006-55-gpt2-01_answer_5.pth'
model_weight_path12 = '../input/photostagea6/best_weights_006-55-gpt2-01_answer_6.pth'



In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torch.nn.utils.weight_norm import weight_norm
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,    GPT2Config,GPT2Model,
    GPT2LMHeadModel,
    GPT2Tokenizer,
)
from transformers.modeling_gpt2 import GPT2PreTrainedModel

In [ ]:
import transformers
print("transformers:", transformers.__version__)

In [ ]:
SEED = 12345

start_time_all = time.time()

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)

In [ ]:
import pickle

def read_pickle_from_file(pickle_file):
    with open(pickle_file,'rb') as f:
        x = pickle.load(f)
    return x

def write_pickle_to_file(pickle_file, x):
    with open(pickle_file, 'wb') as f:
        pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)
        
        
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError

In [ ]:
train_df = pd.read_csv( f"{folder}train.csv" )
train_df.shape
test_df = pd.read_csv( f"{folder}test.csv" )
test_df.shape

In [ ]:
target_table = [
    ['question_asker_intent_understanding',   'question'],
    ['question_body_critical',                'question'],
    ['question_conversational',               'question'],
    ['question_expect_short_answer',          'question'],
    ['question_fact_seeking',                 'question'],
    ['question_has_commonly_accepted_answer', 'question'],
    ['question_interestingness_others',       'question'],
    ['question_interestingness_self',         'question'],
    ['question_multi_intent',                 'question'],
    ['question_not_really_a_question',        'question'],
    ['question_opinion_seeking',              'question'],
    ['question_type_choice',                  'question'],
    ['question_type_compare',                 'question'],
    ['question_type_consequence',             'question'],
    ['question_type_definition',              'question'],
    ['question_type_entity',                  'question'],
    ['question_type_instructions',            'question'],
    ['question_type_procedure',               'question'],
    ['question_type_reason_explanation',      'question'],
    ['question_type_spelling',                'question'],
    ['question_well_written',                 'question'],
    ['answer_helpful',                        'answer'],
    ['answer_level_of_information',           'answer'],
    ['answer_plausible',                      'answer'],
    ['answer_relevance',                      'answer'],
    ['answer_satisfaction',                   'answer'],
    ['answer_type_instructions',              'answer'],
    ['answer_type_procedure',                 'answer'],
    ['answer_type_reason_explanation',        'answer'],
    ['answer_well_written',                   'answer'],    
]

input_columns = [
    'question_title', 
    'question_body',    
    'answer'
]

In [ ]:
target_question_columns = []
target_answer_columns = []

for table in target_table:
    if table[1] == 'question':
        target_question_columns.append( table[0] )
    elif table[1] == 'answer':
        target_answer_columns.append( table[0] )   
        
target_columns = target_question_columns + target_answer_columns

print( 'question:', len(target_question_columns) )
print( 'answer:', len(target_answer_columns) )
print( 'total:', len(target_columns) )

In [ ]:
import html
for df in [train_df, test_df]:

    ## domain components
    df['domcom'] = df['url'].apply(lambda s: s.split('://')[1].split('/')[0].split('.'))

    # count components
    df['dom_cnt'] = df['domcom'].apply(lambda s: len(s))

    # extend length
    df['domcom'] = df['domcom'].apply(lambda s: s + ['none', 'none'])

    # components
    for ii in range(0,4):
        df['url_'+str(ii)] = df['domcom'].apply(lambda s: s[ii])

    # clean up
    df.drop('domcom', axis = 1, inplace = True)
    
    df.question_body = df.question_body.apply(html.unescape)
    df.answer        = df.answer.apply(html.unescape)
    
    df['question_body'] = df['question_body'].apply(lambda s: s.replace("\&gt",">"))
    df['question_body'] = df['question_body'].apply(lambda s: s.replace("\&lt","<"))
    df['question_body'] = df['question_body'].apply(lambda s: s.replace("\&amp","&"))
    df['question_body'] = df['question_body'].apply(lambda s: s.replace("\&quot;","\""))    

In [ ]:
train_feature = pd.DataFrame()
test_feature = pd.DataFrame()
import nltk
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))
for df, feature in [[train_df, train_feature], [test_df, test_feature]]:
    for column in input_columns:
        feature[column+'_total_length'] = df[column].apply(len)
        feature[column+'_capitals'] = df[column].apply(lambda comment: sum(1 for c in comment if c.isupper()))
        feature[column+'_caps_vs_length'] = feature.apply(lambda row: float(row[column+'_capitals'])/float(row[column+'_total_length']),axis=1)
        feature[column+'_num_exclamation_marks'] = df[column].apply(lambda comment: comment.count('!'))
        feature[column+'_num_question_marks'] = df[column].apply(lambda comment: comment.count('?'))
        feature[column+'_num_punctuation'] = df[column].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
        feature[column+'_num_symbols'] = df[column].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
        feature[column+"_num_chars"] = df[column].apply(lambda x: len(str(x)))
        feature[column+'_num_words'] = df[column].apply(lambda comment: len(comment.split()))
        feature[column+'_num_unique_words'] = df[column].apply(lambda comment: len(set(w for w in comment.split())))
        feature[column+'_words_vs_unique'] = feature[column+'_num_unique_words'] / feature[column+'_num_words']
        feature[column+'_num_smilies'] = df[column].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))
        feature[column+'_num_doubt'] = df[column].apply(lambda comment: comment.count('not sure'))
        feature[column+'_num_think'] = df[column].apply(lambda comment: sum(comment.count(w) for w in ['thinking','think','thought']))        
        ## Number of stopwords in the text ##
        feature[column+"_num_stopwords"] = df[column].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
        
        ## Number of punctuations in the text ##
        feature[column+"_num_punctuations"] =df[column].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
        
        ## Number of title case words in the text ##
        feature[column+"_num_words_upper"] = df[column].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
        
nlp = English()
sentencizer = nlp.create_pipe('sentencizer')
nlp.add_pipe(sentencizer)

ans_user_and_category=train_df[train_df[['answer_user_name', 'category']].duplicated()][['answer_user_name', 'category']].values
ans_user_and_category.shape        

In [ ]:
def add_question_metadata_features(text):
    doc=nlp(text)
    indirect = 0
    choice_words=0
    reason_explanation_words = 0
    question_count = 0
    
    for sent in doc.sents:
        if '?' in sent.text and '?' == sent.text[-1]:
            question_count += 1
            for token in sent:
                if token.text.lower()=='why':
                    reason_explanation_words+=1
                elif token.text.lower()=='or':
                    choice_words+=1
    if question_count==0:
        indirect+=1
    
    return np.array([indirect, question_count, reason_explanation_words, choice_words])


def question_answer_author_same(df):
    q_username = df['question_user_name']
    a_username = df['answer_user_name']
    author_same=[]
    
    for i in range(len(df)):
        if q_username[i] == a_username[i]:
            author_same.append(int(1))
        else:
            author_same.append(int(0))
    return author_same


def add_external_features( df, feature ):
    feature['question_vs_answer_length'] = feature['question_body_num_words']/feature['answer_num_words']
    feature['q_a_author_same'] = question_answer_author_same(df)
    
    answer_user_cat = []
    for i in df[['answer_user_name', 'category']].values:
        if i in ans_user_and_category:
            answer_user_cat.append(int(1))
        else:
            answer_user_cat.append(int(0))
    feature['answer_user_cat'] = answer_user_cat
    
    handmade_features=[]
    for text in df['question_body'].values:
        handmade_features.append(add_question_metadata_features(text))

    feature = pd.concat( [ feature, pd.DataFrame( handmade_features, columns=['indirect', 'question_count', 'reason_explanation_words', 'choice_words'])], axis=1 )
    
    return feature

In [ ]:
train_feature = add_external_features( train_df, train_feature )
test_feature  = add_external_features( test_df, test_feature )

In [ ]:
for column in input_columns:
    print( "{} | Min: {}, Max: {}".format( column, train_feature[column+'_total_length'].min(), train_feature[column+'_total_length'].max() ) )
    
print( '=====' )

for column in input_columns:
    print( "{} | Min: {}, Max: {}".format( column, test_feature[column+'_total_length'].min(), test_feature[column+'_total_length'].max() ) )

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
symbol = ["'", '"', ':', ';', '.', ',', '-', '!', '?', "'s", ")", "(", "...", '``', "''", "/", "$", "%", "*", "&", "{", "}", "[", "]"]

def get_prevalent( texts, top_count=15 ):    
    tokenized_sents = [nltk.word_tokenize(i) for i in texts]
    tokenized_sents = [flatten for inner in tokenized_sents for flatten in inner]   
    
    #fdist = nltk.FreqDist(w for w in tokenized_sents if w not in stop_words + symbol)
    fdist = nltk.FreqDist(w.lower() for w in tokenized_sents if w.lower() not in stop_words + symbol)
    comments = fdist.most_common(top_count)
    
    return [word[0] for word in comments]

for column in input_columns:
    words = get_prevalent( train_df[column])
    print( column, words )
    
    for word in words:
        for df, feature in [[train_df, train_feature], [test_df, test_feature]]:
            feature[column+'_num'+word] = df[column].apply(lambda comment: comment.count(word))

In [ ]:
find = re.compile(r"^[^.]*")

train_df['netloc'] = train_df['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test_df['netloc'] = test_df['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

count_columns = ['question_title', 'question_user_name', 'answer_user_name', 'category', 'netloc']

for col in count_columns:
    value = train_df[col].value_counts()
    train_feature[col+'_count'] = train_df[col].map( value )
    test_feature[col+'_count']  = test_df[col].map( value ).fillna( 1 )
    
for col in train_feature.columns: 
    train_mean = np.nanmean( train_feature[col].values )
    train_feature[col].fillna( train_mean, inplace=True )
    test_feature[col].fillna( train_mean, inplace=True ) 
    
print( "train: nan=", np.sum( np.sum( pd.isnull( train_feature ) ) ) )
print( "test : nan=", np.sum( np.sum( pd.isnull( test_feature  ) ) ) )

scaler = MinMaxScaler()
scaler.fit(train_feature)

train_feature = pd.DataFrame(scaler.transform(train_feature), columns=train_feature.columns)
test_feature = pd.DataFrame(scaler.transform(test_feature), columns=test_feature.columns)

del scaler
gc.collect()

In [ ]:
print( train_feature.shape )
print( test_feature.shape )
print( 'time: {}'.format( time_to_str((time.time()-start_time_all),'min') ) )

In [ ]:
def convert_label( df, result_df, column, count=10 ):
    labels = [ ( df[column].values>=(rate/count) ).astype(np.int) for rate in range( 1, count+1 ) ]

    columns = ['{}_{}'.format(column, i) for i in range(count)]
    labels = np.array( labels ).T
    
    label_df = pd.DataFrame( labels, columns=columns )
    result_df = pd.concat((result_df, label_df), axis=1)
    
    return result_df


def convert_label_origin( df, result_df, column, count=10 ):
    
    columns = ['{}_{}'.format(column, i) for i in range(count)]
    labels = df[columns].values
    values = []
    
    for i in range( len(labels ) ):
        value = 0.0
                   
        for j in range(count):
            if labels[i][j] > 0.5:
                value = (j+1) / count
    
        values.append( value )
                   
    label_df = pd.DataFrame( values, columns=[column] )
    result_df = pd.concat((result_df, label_df), axis=1)
    
    return result_df

In [ ]:
label_convert_count = 12
train_feature2 = pd.DataFrame()
test_feature2 = pd.DataFrame()

for column in train_feature.columns:
    train_feature2 = convert_label( train_feature, train_feature2, column, label_convert_count )
    test_feature2 = convert_label( test_feature, test_feature2, column, label_convert_count )   
    
train_feature = train_feature2.copy()
test_feature = test_feature2.copy()

del train_feature2, test_feature2

In [ ]:
print( train_feature.shape )
print( test_feature.shape )

In [ ]:
features = ['question_title', 'question_user_name', 'answer_user_name']
limits = [6, 8, 8]

#features = ['question_title', 'question_user_name', 'answer_user_name', 'category', 'url_0', 'url_1']
#limits = [6, 8, 8, 1, 60, 1]

for col, limit in zip( features, limits ):
    value = train_df[col].value_counts()
    train_df['item_count'] = train_df[col].map( value )    
    train_df['item_value'] = train_df[col].copy()
    train_df.loc[train_df.item_count < limit, 'item_value'] = "limit_abcdefg123456789"
    
    test_df['item_count'] = test_df[col].map( value ).fillna( 1 )    
    test_df['item_value'] = test_df[col].copy()
    test_df.loc[test_df.item_count < limit, 'item_value'] = "limit_abcdefg123456789"    
    
    lb = LabelBinarizer()
    lb.fit( train_df['item_value'] )
    
    encode_train = lb.transform(train_df['item_value'])
    encode_test = lb.transform(test_df['item_value'])
    
    columns = ['LabelBinarizer_{}'.format(i) for i in range(encode_train.shape[1])]
    print( "{}: {}". format( col, len( train_df['item_value'].value_counts() ) ) )
    
    encode_train = pd.DataFrame( encode_train, columns=columns )
    train_feature = pd.concat((train_feature, encode_train), axis=1)

    encode_test = pd.DataFrame( encode_test, columns=columns )
    test_feature = pd.concat((test_feature, encode_test), axis=1)      
    
    del lb
    
train_df.drop( ['item_count', 'item_count'], axis=1, inplace=True )
test_df.drop( ['item_count', 'item_count'], axis=1, inplace=True )

In [ ]:
features = ['url_0', 'category']
enc = OneHotEncoder( handle_unknown='ignore' )
enc.fit( train_df[features] )

encode_train = enc.transform(train_df[features]).toarray()
encode_test = enc.transform(test_df[features]).toarray()

columns = ['encode_{}'.format(i) for i in range(encode_train.shape[1])]

encode_train = pd.DataFrame( encode_train, columns=columns )
train_feature = pd.concat((train_feature, encode_train), axis=1)

encode_test = pd.DataFrame( encode_test, columns=columns )
test_feature = pd.concat((test_feature, encode_test), axis=1)   

del encode_train, encode_test, enc
print( train_feature.shape )
print( test_feature.shape )
print( 'time: {}'.format( time_to_str((time.time()-start_time_all),'min') ) )

In [ ]:
for col in train_feature.columns: 
    train_mean = np.nanmean( train_feature[col].values )
    train_feature[col].fillna( train_mean, inplace=True )
    test_feature[col].fillna( train_mean, inplace=True ) 
    
print( "train: nan=", np.sum( np.sum( pd.isnull( train_feature ) ) ) )
print( "test : nan=", np.sum( np.sum( pd.isnull( test_feature  ) ) ) )


In [ ]:
# https://www.kaggle.com/chenshengabc/from-quest-encoding-ensemble-a-little-bit-differen

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
#         df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
        df[col] = df[col].apply(lambda x: handle_contractions(x))  
        df[col] = df[col].apply(lambda x: fix_quote(x))   
    
    return df


train_df = clean_data( train_df, input_columns )
test_df = clean_data( test_df, input_columns )

del tokenizer

In [ ]:
# 'Cyclical Learning Rates for Training Neural Networks'- Leslie N. Smith, arxiv 2017
#       https://arxiv.org/abs/1506.01186
#       https://github.com/bckenstler/CLR

class CyclicScheduler1():

    def __init__(self, min_lr=0.001, max_lr=0.01, period=10 ):
        super(CyclicScheduler1, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.period = period

    def __call__(self, time):

        #sawtooth
        #r = (1-(time%self.period)/self.period)

        #cosine
        time= time%self.period
        r = (np.cos(time/self.period *np.pi)+1)/2

        lr = self.min_lr + r*(self.max_lr-self.min_lr)
        return lr

    def __str__(self):
        string = 'CyclicScheduler\n' \
                + 'min_lr=%0.3f, max_lr=%0.3f, period=%8.1f'%(self.min_lr, self.max_lr, self.period)
        return string


class CyclicScheduler2():

    def __init__(self, min_lr=0.001, max_lr=0.01, period=10, max_decay=1.0, warm_start=0 ):
        super(CyclicScheduler2, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.period = period
        self.max_decay = max_decay
        self.warm_start = warm_start
        self.cycle = -1

    def __call__(self, time):
        if time<self.warm_start: return self.max_lr

        #cosine
        self.cycle = (time-self.warm_start)//self.period
        time = (time-self.warm_start)%self.period

        period = self.period
        min_lr = self.min_lr
        max_lr = self.max_lr *(self.max_decay**self.cycle)


        r   = (np.cos(time/period *np.pi)+1)/2
        lr = min_lr + r*(max_lr-min_lr)

        return lr



    def __str__(self):
        string = 'CyclicScheduler\n' \
                + 'min_lr=%0.4f, max_lr=%0.4f, period=%8.1f'%(self.min_lr, self.max_lr, self.period)
        return string


#tanh curve
class CyclicScheduler3():

    def __init__(self, min_lr=0.001, max_lr=0.01, period=10, max_decay=1.0, warm_start=0 ):
        super(CyclicScheduler3, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.period = period
        self.max_decay = max_decay
        self.warm_start = warm_start
        self.cycle = -1

    def __call__(self, time):
        if time<self.warm_start: return self.max_lr

        #cosine
        self.cycle = (time-self.warm_start)//self.period
        time = (time-self.warm_start)%self.period

        period = self.period
        min_lr = self.min_lr
        max_lr = self.max_lr *(self.max_decay**self.cycle)

        r   = (np.tanh(-time/period *16 +8)+1)*0.5
        lr = min_lr + r*(max_lr-min_lr)

        return lr



    def __str__(self):
        string = 'CyclicScheduler\n' \
                + 'min_lr=%0.3f, max_lr=%0.3f, period=%8.1f'%(self.min_lr, self.max_lr, self.period)
        return string


class NullScheduler():
    def __init__(self, lr=0.01 ):
        super(NullScheduler, self).__init__()
        self.lr    = lr
        self.cycle = 0

    def __call__(self, time):
        return self.lr

    def __str__(self):
        string = 'NullScheduler\n' \
                + 'lr=%0.5f '%(self.lr)
        return string


# net ------------------------------------
# https://github.com/pytorch/examples/blob/master/imagenet/main.py ###############
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

In [ ]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
class NeuralNet3(nn.Module):
    def __init__(self, num_features, num_labels, pretrained_bert):
        super(NeuralNet3, self).__init__()
        self.config = transformers.GPT2Config.from_pretrained(pretrained_bert, output_hidden_states=False)
        self.gptmodel = transformers.GPT2Model(self.config)
        self.gpt = self.gptmodel.from_pretrained( pretrained_bert,config =self.config)
        
        self.num_features = num_features
        self.num_labels = num_labels
        
        self.encoded_dropout = SpatialDropout( 0.2 )
        self.pooled_dropout = nn.Dropout( 0.2 )        
        
        self.feature_linear = nn.Sequential(
            nn.Linear( self.num_features, self.num_features ),
            nn.ReLU( inplace=True ),
            nn.Dropout( 0.2 ),            
        )
        
        '''
        self.feature_linear = nn.Sequential(
            nn.Linear( self.num_features, self.num_features ),
            nn.BatchNorm1d( self.num_features ),
            nn.ReLU( inplace=True ),
            nn.Linear( self.num_features, self.num_features2 ),
        )
        '''
        
        dense_hidden_units = self.gpt.config.hidden_size * 3 + self.num_features
        
        self.linear1 = nn.Linear( dense_hidden_units, dense_hidden_units )
        self.linear2 = nn.Linear( dense_hidden_units, dense_hidden_units )
        
        self.classifier = nn.Sequential(
            nn.Linear( dense_hidden_units, num_labels ),
        )
        
    def forward( self, ids, masks, segments, feature ):  
        
        feature_output = self.feature_linear( feature )
        outputs = self.gpt( input_ids=ids, attention_mask=masks, token_type_ids=segments)

        last_hidden_state,present = self.gpt( input_ids=ids, attention_mask=masks, token_type_ids=segments)
       # sequence_output = self.encoded_dropout(sequence_output)
       # pooled_output = self.pooled_dropout(pooled_output)     

      #  h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
      #  h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
      #  h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
      #  h9 = hidden_states[-4][:, 0].reshape((-1, 1, 768))

      #  hidden_states = torch.cat([h9, h10, h11, h12], 1)
        last_hidden_state = self.encoded_dropout(last_hidden_state)

        avg_pool = torch.mean( last_hidden_state, 1 )
        max_pool, _ = torch.max( last_hidden_state, 1 )

        h_conc = torch.cat( ( avg_pool, max_pool, last_hidden_state[:, -1, :], feature_output ), 1 )
        h_conc_linear  = F.relu(self.linear1(h_conc))
        hidden = h_conc + h_conc_linear 
            
        h_conc_linear  = F.relu(self.linear2(hidden))
        hidden = hidden + h_conc_linear      
        
        return self.classifier( hidden )
    

In [ ]:
def compute_input_title_question( df, tokenizer, max_sequence_length=512 ):
    
    input_ids, input_masks, input_segments = [], [], []
    
    for _, instance in df.iterrows():
         
        title    = tokenizer.tokenize(instance.question_title)
        question = tokenizer.tokenize(instance.question_body)

        if (len(title)+len(question)+3) > max_sequence_length:
            if len(title) > 30:
                title = title[:30]
                
            question_len = max_sequence_length - len(title) - 3
            question = question[:question_len]
        
        #token = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] 
        #token_ids = tokenizer.convert_tokens_to_ids(token)
        
        title_ids    = tokenizer.convert_tokens_to_ids(title)
        question_ids = tokenizer.convert_tokens_to_ids(question)
        cls_ids      = tokenizer.convert_tokens_to_ids( ["[CLS]"] )   
        sep_ids      = tokenizer.convert_tokens_to_ids( ["[SEP]"] ) 
        
        token_ids =  title_ids  + question_ids 
        padding = [0] * (max_sequence_length - len(token_ids))
        
        ids      = token_ids + padding
        masks    = [1]*len(token_ids) + padding
        len_q = len(question_ids)
        len_qf = int(len(question_ids)/2)
        len_ql = len_q-len_qf
        segments = [0]*(len(title_ids)) + [1]*(len(question_ids)) + padding
       # segments = [0]*(len(title_ids)+2) + [1]*(len_qf+1)+[2]*(len_ql) + padding

        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),   
    ]

In [ ]:
def compute_input_title_answer( df, tokenizer, max_sequence_length=512 ):
    
    input_ids, input_masks, input_segments = [], [], []
    
    for _, instance in df.iterrows():
         
        title  = tokenizer.tokenize(instance.question_title)
        answer = tokenizer.tokenize(instance.answer)

        if (len(title)+len(answer)+3) > max_sequence_length:
            if len(title) > 30:
                title = title[:30]
                
            answer_len = max_sequence_length - len(title) - 3
            answer = answer[:answer_len]
        
        #token = ["[CLS]"] + title + ["[SEP]"] + answer + ["[SEP]"] 
        #token_ids = tokenizer.convert_tokens_to_ids(token)
        
        title_ids  = tokenizer.convert_tokens_to_ids(title)
        answer_ids = tokenizer.convert_tokens_to_ids(answer)
        cls_ids    = tokenizer.convert_tokens_to_ids( ["[CLS]"] )   
        sep_ids    = tokenizer.convert_tokens_to_ids( ["[SEP]"] ) 
        
        token_ids =  title_ids  + answer_ids 
        padding = [0] * (max_sequence_length - len(token_ids))
        
        ids      = token_ids + padding
        masks    = [1]*len(token_ids) + padding
        segments = [0]*(len(title_ids)) + [1]*(len(answer_ids)) + padding
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),   
    ]

In [ ]:
def compute_input_arrays( df, tokenizer, max_sequence_length=512 ):
    
    input_ids, input_masks, input_segments = [], [], []    
    t_max_len = 35     
    
    for _, instance in df.iterrows():
        
        title    = tokenizer.tokenize(instance.question_title)
        question = tokenizer.tokenize(instance.question_body)
        answer   = tokenizer.tokenize(instance.answer)

        if (len(title)+len(question)+len(answer)+4) > max_sequence_length:
            if len(title) > t_max_len:
                title = title[:t_max_len]
                
            question_len = ( max_sequence_length - len(title) - 4 ) // 2
            question = question[:question_len]
            
            answer_len = max_sequence_length - len(title) - len(question) - 4
            answer = answer[:answer_len]
        
        #token = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
        #token_ids = tokenizer.convert_tokens_to_ids(token)
        
        title_ids    = tokenizer.convert_tokens_to_ids(title)
        question_ids = tokenizer.convert_tokens_to_ids(question)
        answer_ids   = tokenizer.convert_tokens_to_ids(answer) 
        cls_ids      = tokenizer.convert_tokens_to_ids( ["[CLS]"] )   
        sep_ids      = tokenizer.convert_tokens_to_ids( ["[SEP]"] ) 
        
        token_ids = cls_ids + title_ids + sep_ids + question_ids + sep_ids + answer_ids + sep_ids
        padding = [0] * (max_sequence_length - len(token_ids))
        
        ids      = token_ids + padding
        masks    = [1]*len(token_ids) + padding
        segments = [0]*(len(title_ids)+len(question_ids)+3) + [1]*(len(answer_ids)+1) + padding
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),   
    ]

In [ ]:
from sklearn.model_selection import GroupKFold

In [ ]:
n_splits = 6

'''
x_train = train_feature.values
y_train = train_df[target_columns].values

cv = KFold( n_splits=n_splits, random_state=SEED )
kfold_split = list( cv.split( x_train, y_train ) )'''

cv = GroupKFold( n_splits=n_splits )
kfold_split = list( cv.split( X=train_df.question_body, groups=train_df.question_body ) )   

'''write_pickle_to_file( 'kfold_split_index.pkl', kfold_split )
'''

#kfold_split = read_pickle_from_file( model_weight_path1 + 'kfold_split_index.pkl' )


In [ ]:
test_pred_datas = {}
test_pred_weights = {}

for column in target_columns:
    test_pred_datas[column] = np.zeros( len(test_df) )
    test_pred_weights[column] = 0.0
    
def add_test_pred_data( prediction, columns, weight ):
    for column_idx, column in enumerate( columns ):
        test_pred_datas[column] += weight * prediction[:, column_idx]  
        test_pred_weights[column] += weight    
        
validation_datas = {}
validation_counts = {}

for column in target_columns:
    validation_datas[column] = np.zeros( len(train_df) )
    validation_counts[column] = np.zeros( len(train_df) )

def add_validation_data( prediction, columns, idx ):
    for column_idx, column in enumerate( columns ):
        validation_datas[column][idx] += prediction[:, column_idx]  
        validation_counts[column][idx] += 1.0    
        
        
def mean_spearmanr_correlation_score( y_true, y_pred ):
    num_labels = y_pred.shape[1]
    score = np.nanmean( [ spearmanr( y_pred[:, idx], y_true[:, idx] ).correlation for idx in range(num_labels) ] )
    return score 

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, features, labels = None):
        
        self.inputs   = inputs
        self.features = features
        
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        input_features  = self.features[idx]
        
        if self.labels is not None: # targets
            input_labels = self.labels[idx]
            return input_ids, input_masks, input_segments, input_features, input_labels
        
        return input_ids, input_masks, input_segments, input_features

    def __len__(self):
        return len(self.inputs[0])

In [ ]:
def model_test( test_inputs, x_test, label_columns, weight_files ):
    
    batch_size = 6

    test_dataset = QuestDataset( test_inputs, x_test, None )
    test_loader = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size, shuffle=False )    
    
    num_features = x_test.shape[1]
    num_labels   = len(label_columns)

    for fname in weight_files:

        model = NeuralNet3( num_features, num_labels, pretrained_bert )
        model.cuda()

        model.load_state_dict( torch.load( fname ) )      
        model.eval()
     
        test_preds_fold  = []

        with torch.no_grad():
            for ids, masks, segments, features in test_loader:
                ids      = ids.cuda()
                masks    = masks.cuda()
                segments = segments.cuda()
                features = torch.tensor(features,dtype=torch.float32).cuda()
                y_pred = model( ids, masks, segments, features )
                test_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

            test_preds_fold = np.array( test_preds_fold )
            add_test_pred_data( test_preds_fold, label_columns, 1.0 )

        del model
        torch.cuda.empty_cache()
        gc.collect()
        
    del test_dataset, test_loader

In [ ]:
def model_validation( train_inputs, x_train, label_columns, weight_files ):
    
    if len(kfold_split) != len(weight_files):
        return
    
    batch_size   = 6    
    num_features = x_test.shape[1]
    num_labels   = len(label_columns)

    for k, (train_idx, valid_idx) in enumerate(kfold_split):
        
        x_train_valid = x_train[valid_idx]

        train_inputs_valid = [x[valid_idx] for x in train_inputs]

        valid_dataset = QuestDataset( train_inputs_valid, x_train_valid, None )
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False )
        
        model = NeuralNet3( num_features, num_labels, pretrained_bert )
        model.cuda()

        fname = weight_files[k]
        model.load_state_dict( torch.load( fname ) )      
        model.eval()
     
        valid_preds_fold  = []

        with torch.no_grad():
            for ids, masks, segments, features in valid_loader:
                ids      = ids.cuda()
                masks    = masks.cuda()
                segments = segments.cuda()
                features = torch.tensor(features,dtype=torch.float32).cuda()
                y_pred = model( ids, masks, segments, features )
                valid_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

            valid_preds_fold = np.array( valid_preds_fold )
            add_validation_data( valid_preds_fold, label_columns, valid_idx )

        del model, valid_dataset, valid_loader
        torch.cuda.empty_cache()
        gc.collect()

In [ ]:
def model_test_validation( label_columns, train_inputs, test_inputs, x_train, x_test, weight_files ):
    
    if len(kfold_split) != len(weight_files):
        return
    
    batch_size   = 6    
    num_features = x_test.shape[1]
    num_labels   = len(label_columns)

    test_dataset = QuestDataset( test_inputs, x_test, None )
    test_loader = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size, shuffle=False )   
    
    for k, (train_idx, valid_idx) in enumerate(kfold_split):
        
        fname = weight_files[k]
        print( k+1, fname )
        
        x_train_valid = x_train[valid_idx]
        train_inputs_valid = [x[valid_idx] for x in train_inputs]

        valid_dataset = QuestDataset( train_inputs_valid, x_train_valid, None )
        valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False )
        
        model = NeuralNet3( num_features, num_labels, pretrained_bert )
        model.cuda()

        model.load_state_dict( torch.load( fname ) )      
        model.eval()
     
        #====================
        #validation
        valid_preds_fold  = []

        with torch.no_grad():
            for ids, masks, segments, features in valid_loader:
                ids      = ids.cuda()
                masks    = masks.cuda()
                segments = segments.cuda()
                features = torch.tensor(features,dtype=torch.float32).cuda()
                y_pred = model( ids, masks, segments, features )
                valid_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

            valid_preds_fold = np.array( valid_preds_fold )
            add_validation_data( valid_preds_fold, label_columns, valid_idx )
            
        #====================
        #test
        test_preds_fold  = []

        with torch.no_grad():
            for ids, masks, segments, features in test_loader:
                ids      = ids.cuda()
                masks    = masks.cuda()
                segments = segments.cuda()
                features = torch.tensor(features,dtype=torch.float32).cuda()
                y_pred = model( ids, masks, segments, features )
                test_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

            test_preds_fold = np.array( test_preds_fold )
            add_test_pred_data( test_preds_fold, label_columns, 1.0 )            

        del model, valid_dataset, valid_loader
        torch.cuda.empty_cache()
        gc.collect()
        
    del test_dataset, test_loader

In [ ]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained( pretrained_bert )

In [ ]:
print( 'time: {}'.format( time_to_str((time.time()-start_time_all), 'min' ) ) )

In [ ]:
train_inputs = compute_input_title_question( train_df, tokenizer, max_sequence_length=512 )
test_inputs = compute_input_title_question( test_df, tokenizer, max_sequence_length=512 )

x_train = train_feature.values
x_test  = test_feature.values

print( x_train.shape )
print( x_test.shape )
print( len( target_question_columns ) )

weight_files = [
    model_weight_path1 ,
    model_weight_path2 ,
    model_weight_path3 ,
    model_weight_path4 ,
    model_weight_path5,
    model_weight_path6,
]

model_test_validation( target_question_columns, train_inputs, test_inputs, x_train, x_test, weight_files )

del x_train, x_test
del train_inputs, test_inputs


In [ ]:
train_inputs = compute_input_arrays( train_df, tokenizer, max_sequence_length=512 )
test_inputs = compute_input_arrays( test_df, tokenizer, max_sequence_length=512 )

x_train = train_feature.values
x_test  = test_feature.values

print( x_train.shape )
print( x_test.shape )
print( len( target_answer_columns ) )

weight_files = [
    model_weight_path7,
    model_weight_path8,
    model_weight_path9,
    model_weight_path10,
    model_weight_path11,
    model_weight_path12,
]

model_test_validation( target_answer_columns, train_inputs, test_inputs, x_train, x_test, weight_files )

del x_train, x_test
del train_inputs, test_inputs


In [ ]:
del train_feature, test_feature
del target_question_columns, target_answer_columns 


In [ ]:
for column in target_columns:
    print( np.sum( validation_counts[column] ), np.min( validation_counts[column] ), np.max( validation_counts[column] ), column )
    
validationTG  = pd.DataFrame()

for column in target_columns:
    preds = validation_datas[column]
    count = validation_counts[column]
    count = np.where( count < 0.5, 1.0, count )
    
    validationTG[column] = preds / count
    
mean_spearmanr_correlation_score( validationTG.values, train_df[target_columns].values )   
validationTG.head()

In [ ]:
mean_spearmanr_correlation_score( validationTG.values, train_df[target_columns].values )   

In [ ]:
test_predsTG = pd.read_csv( f"{folder}sample_submission.csv" )

for column in target_columns:
    preds = test_pred_datas[column]
    weight = test_pred_weights[column]
    test_predsTG[column] = preds / weight    
    #output = rankdata( output )
    #max_val = np.max(output) + 1
    #output = output / max_val + 1e-12
        
test_predsTG.head()

In [ ]:
del target_columns, input_columns
del test_pred_datas, test_pred_weights
del validation_datas, validation_counts

torch.cuda.empty_cache()
gc.collect()

# QUEST_005_04_1

# QUEST_005_07_DISTILBERT+LSTM (DISTILBERT)

In [ ]:
folder = '../input/google-quest-challenge/'
pretrained_distilbert_base_uncased = "../input/pretrainedbertpytorch/pretrained-bert-pytorch/distilbert-base-uncased/"
universal_sentence_encoder_path = "../input/universalsentenceencoderlarge4/"

In [ ]:
import os
import re
import gc

import pickle  
import numpy as np
import pandas as pd
import random
import copy
import string
import time

import nltk
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.model_selection import KFold, GroupKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import math
from tqdm import tqdm

from spacy.lang.en import English
from urllib.parse import urlparse
import math

import warnings
warnings.simplefilter('ignore')

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from torch.nn.utils.weight_norm import weight_norm

from scipy.stats import spearmanr

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
import transformers
print("transformers:", transformers.__version__)

In [ ]:
from radam import RAdam

In [ ]:
SEED = 12345

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)

In [ ]:
import pickle

def read_pickle_from_file(pickle_file):
    with open(pickle_file,'rb') as f:
        x = pickle.load(f)
    return x

def write_pickle_to_file(pickle_file, x):
    with open(pickle_file, 'wb') as f:
        pickle.dump(x, f, pickle.HIGHEST_PROTOCOL)

## Load data

In [ ]:
train_df = pd.read_csv( f"{folder}train.csv" )
train_df.shape

In [ ]:
test_df = pd.read_csv( f"{folder}test.csv" )
test_df.shape

In [ ]:
#train_df.head(3).T

## Extract target variables

In [ ]:
target_columns = [
    'question_asker_intent_understanding',
    'question_body_critical',
    'question_conversational',
    'question_expect_short_answer',
    'question_fact_seeking',
    'question_has_commonly_accepted_answer',
    'question_interestingness_others',
    'question_interestingness_self',
    'question_multi_intent',
    'question_not_really_a_question',
    'question_opinion_seeking',
    'question_type_choice',
    'question_type_compare',
    'question_type_consequence',
    'question_type_definition',
    'question_type_entity',
    'question_type_instructions',
    'question_type_procedure',
    'question_type_reason_explanation',
    'question_type_spelling',
    'question_well_written',
    'answer_helpful',
    'answer_level_of_information',
    'answer_plausible',
    'answer_relevance',
    'answer_satisfaction',
    'answer_type_instructions',
    'answer_type_procedure',
    'answer_type_reason_explanation',
    'answer_well_written'    
]

input_columns = [
    'question_title', 
    'question_body',    
    'answer'
]

In [ ]:
print( 'target_columns:', len(target_columns) )
print( 'input_columns:', len(input_columns) )

In [ ]:
for df in [train_df, test_df]:

    ## domain components
    df['domcom'] = df['url'].apply(lambda s: s.split('://')[1].split('/')[0].split('.'))

    # count components
    df['dom_cnt'] = df['domcom'].apply(lambda s: len(s))

    # extend length
    df['domcom'] = df['domcom'].apply(lambda s: s + ['none', 'none'])

    # components
    for ii in range(0,4):
        df['url_'+str(ii)] = df['domcom'].apply(lambda s: s[ii])

    # clean up
    df.drop('domcom', axis = 1, inplace = True)

## Features

In [ ]:
train_feature = pd.DataFrame()
test_feature = pd.DataFrame()

In [ ]:
features = ['url_0', 'category']
enc = OneHotEncoder( handle_unknown='ignore' )
enc.fit( train_df[features] )

encode_train = enc.transform(train_df[features]).toarray()
encode_test = enc.transform(test_df[features]).toarray()

columns = ['encode_{}'.format(i) for i in range(encode_train.shape[1])]

encode_train = pd.DataFrame( encode_train, columns=columns )
train_feature = pd.concat((train_feature, encode_train), axis=1)

encode_test = pd.DataFrame( encode_test, columns=columns )
test_feature = pd.concat((test_feature, encode_test), axis=1)   

del encode_train, encode_test, enc

In [ ]:
print( train_feature.shape )
print( test_feature.shape )

### DISTILBERT

In [ ]:
class QuestBertDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels = None):
        
        self.inputs   = inputs
        
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        
        if self.labels is not None: # targets
            input_labels = self.labels[idx]
            return input_ids, input_masks, input_labels
        
        return input_ids, input_masks

    def __len__(self):
        return len(self.inputs[0])

In [ ]:
def compute_input_text( texts, tokenizer, max_sequence_length=512 ):
    
    input_ids   = []
    input_masks = []
    
    for text in texts:
         
        text = tokenizer.tokenize(text)
        
        text_ids = tokenizer.convert_tokens_to_ids(text)
        cls_ids  = tokenizer.convert_tokens_to_ids( ["[CLS]"] )   
        sep_ids  = tokenizer.convert_tokens_to_ids( ["[SEP]"] ) 
        
        if (len(text_ids)+2) > max_sequence_length:
            text_ids = text_ids[:max_sequence_length-2]        
        
        token_ids = cls_ids + text_ids + sep_ids
        padding = [0] * (max_sequence_length - len(token_ids))
        
        ids      = token_ids + padding
        masks    = [1]*len(token_ids) + padding
        
        input_ids.append(ids)
        input_masks.append(masks)

    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long()
    ]

In [ ]:
def get_bert_feature( tokenizer, model, df, column ):
    
    print( column )
    
    inputs = compute_input_text( df[column].values, tokenizer )
    dataset = QuestBertDataset( inputs, None )
    loader = torch.utils.data.DataLoader( dataset, batch_size=6, shuffle=False, drop_last=False )   
    
    preds_fold = []

    model.eval() 

    with torch.no_grad():
        for ids, masks in tqdm(loader):
            ids   = ids.cuda()
            masks = masks.cuda()

            outputs = model( input_ids=ids, attention_mask=masks )  
            x = outputs[0][:, 0, :]

            preds_fold.extend( x.cpu().data.numpy().tolist() )    
            
    preds_fold = np.array( preds_fold )
    
    return preds_fold

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained( pretrained_distilbert_base_uncased )
model = transformers.DistilBertModel.from_pretrained( pretrained_distilbert_base_uncased )
model.cuda()

print( 'train:' )
train_question_title_dense = get_bert_feature( tokenizer, model, train_df, 'question_title' )
train_question_body_dense  = get_bert_feature( tokenizer, model, train_df, 'question_body' )
train_answer_dense         = get_bert_feature( tokenizer, model, train_df, 'answer' )

print( 'test:' )
test_question_title_dense = get_bert_feature( tokenizer, model, test_df, 'question_title' )
test_question_body_dense  = get_bert_feature( tokenizer, model, test_df, 'question_body' )
test_answer_dense         = get_bert_feature( tokenizer, model, test_df, 'answer' )

del tokenizer, model

### Universal Sentence Encoder

In [ ]:
embed = hub.load(universal_sentence_encoder_path)

embeddings_train = {}
embeddings_test = {}

for text in input_columns:
    print(text)
    train_text = train_df[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = test_df[text].str.replace('?', '.').str.replace('!', '.').tolist()
    
    curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
    ind = 0
    while ind*batch_size < len(train_text):
        curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1
        
    ind = 0
    while ind*batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1    
        
    embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)
    
del embed
gc.collect()

In [ ]:
embeddings_train_df = pd.DataFrame()
embeddings_test_df = pd.DataFrame()

l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)
cos_dist = lambda x, y: (x*y).sum(axis=1)
abs_dist = lambda x, y: np.abs(x-y).sum(axis=1)
sum_dist = lambda x, y: (x+y).sum(axis=1)

dist_columns = [
    ['question_title_embedding', 'answer_embedding'],
    ['question_body_embedding', 'answer_embedding'],
    ['question_body_embedding', 'question_title_embedding'],
]

for i, columns in enumerate(dist_columns):
    embeddings_train_df[f'l2_dist_embedding_{i}']  = l2_dist( embeddings_train[columns[0]], embeddings_train[columns[1]] )
    embeddings_train_df[f'cos_dist_embedding_{i}'] = cos_dist( embeddings_train[columns[0]], embeddings_train[columns[1]] )
    embeddings_train_df[f'abs_dist_embedding_{i}'] = abs_dist( embeddings_train[columns[0]], embeddings_train[columns[1]] )
    embeddings_train_df[f'l2_dist_embedding_{i}']  = sum_dist( embeddings_train[columns[0]], embeddings_train[columns[1]] )
    
    embeddings_test_df[f'l2_dist_embedding_{i}']  = l2_dist( embeddings_test[columns[0]], embeddings_test[columns[1]] )
    embeddings_test_df[f'cos_dist_embedding_{i}'] = cos_dist( embeddings_test[columns[0]], embeddings_test[columns[1]] )
    embeddings_test_df[f'abs_dist_embedding_{i}'] = abs_dist( embeddings_test[columns[0]], embeddings_test[columns[1]] )
    embeddings_test_df[f'l2_dist_embedding_{i}']  = sum_dist( embeddings_test[columns[0]], embeddings_test[columns[1]] )    

In [ ]:
x_train = np.hstack( [item for k, item in embeddings_train.items()] + [train_question_title_dense, train_question_body_dense, train_answer_dense, embeddings_train_df.values, train_feature.values]  )
x_test  = np.hstack( [item for k, item in embeddings_test.items()] + [test_question_title_dense, test_question_body_dense, test_answer_dense, embeddings_test_df.values, test_feature.values]  )
y_train = train_df[target_columns].values

In [ ]:
del train_feature, test_feature
del embeddings_train, embeddings_test
del embeddings_train_df, embeddings_test_df
del train_question_title_dense, train_question_body_dense, train_answer_dense
del test_question_title_dense, test_question_body_dense, test_answer_dense

In [ ]:
print( x_train.shape )
print( x_test.shape )
print( y_train.shape )

### Model

In [ ]:
# 'Cyclical Learning Rates for Training Neural Networks'- Leslie N. Smith, arxiv 2017
#       https://arxiv.org/abs/1506.01186
#       https://github.com/bckenstler/CLR

class CyclicScheduler1():

    def __init__(self, min_lr=0.001, max_lr=0.01, period=10 ):
        super(CyclicScheduler1, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.period = period

    def __call__(self, time):

        #sawtooth
        #r = (1-(time%self.period)/self.period)

        #cosine
        time= time%self.period
        r = (np.cos(time/self.period *np.pi)+1)/2

        lr = self.min_lr + r*(self.max_lr-self.min_lr)
        return lr

    def __str__(self):
        string = 'CyclicScheduler\n' \
                + 'min_lr=%0.3f, max_lr=%0.3f, period=%8.1f'%(self.min_lr, self.max_lr, self.period)
        return string


class CyclicScheduler2():

    def __init__(self, min_lr=0.001, max_lr=0.01, period=10, max_decay=1.0, warm_start=0 ):
        super(CyclicScheduler2, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.period = period
        self.max_decay = max_decay
        self.warm_start = warm_start
        self.cycle = -1

    def __call__(self, time):
        if time<self.warm_start: return self.max_lr

        #cosine
        self.cycle = (time-self.warm_start)//self.period
        time = (time-self.warm_start)%self.period

        period = self.period
        min_lr = self.min_lr
        max_lr = self.max_lr *(self.max_decay**self.cycle)


        r   = (np.cos(time/period *np.pi)+1)/2
        lr = min_lr + r*(max_lr-min_lr)

        return lr



    def __str__(self):
        string = 'CyclicScheduler\n' \
                + 'min_lr=%0.4f, max_lr=%0.4f, period=%8.1f'%(self.min_lr, self.max_lr, self.period)
        return string


#tanh curve
class CyclicScheduler3():

    def __init__(self, min_lr=0.001, max_lr=0.01, period=10, max_decay=1.0, warm_start=0 ):
        super(CyclicScheduler3, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.period = period
        self.max_decay = max_decay
        self.warm_start = warm_start
        self.cycle = -1

    def __call__(self, time):
        if time<self.warm_start: return self.max_lr

        #cosine
        self.cycle = (time-self.warm_start)//self.period
        time = (time-self.warm_start)%self.period

        period = self.period
        min_lr = self.min_lr
        max_lr = self.max_lr *(self.max_decay**self.cycle)

        r   = (np.tanh(-time/period *16 +8)+1)*0.5
        lr = min_lr + r*(max_lr-min_lr)

        return lr



    def __str__(self):
        string = 'CyclicScheduler\n' \
                + 'min_lr=%0.3f, max_lr=%0.3f, period=%8.1f'%(self.min_lr, self.max_lr, self.period)
        return string


class NullScheduler():
    def __init__(self, lr=0.01 ):
        super(NullScheduler, self).__init__()
        self.lr    = lr
        self.cycle = 0

    def __call__(self, time):
        return self.lr

    def __str__(self):
        string = 'NullScheduler\n' \
                + 'lr=%0.5f '%(self.lr)
        return string


# net ------------------------------------
# https://github.com/pytorch/examples/blob/master/imagenet/main.py ###############
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

### Model

In [ ]:
def mean_spearmanr_correlation_score( y_true, y_pred ):
    num_labels = y_pred.shape[1]
    return np.nanmean( [ spearmanr( y_pred[:, idx], y_true[:, idx] ).correlation for idx in range(num_labels) ] )

In [ ]:
class QuestDataset( torch.utils.data.Dataset ):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.x[idx], self.y[idx]
        else:
            return self.x[idx]

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, num_features, num_labels):
        super(NeuralNet, self).__init__()
        
        self.num_features = num_features
        self.num_labels = num_labels
        
        self.classifier = nn.Sequential(
            nn.Linear( self.num_features, self.num_features ),
            nn.Dropout( 0.2 ),
            nn.Linear( self.num_features, self.num_features),
          #  nn.Dropout( 0.1 ),                 
            nn.Linear( self.num_features, self.num_labels ),
        )
        
    def forward( self, features ):
        return self.classifier( features )

In [ ]:
test_pred_datas = {}
test_pred_weights = {}

def init_test_pred_data():
    for column in target_columns:
        test_pred_datas[column] = np.zeros( len(test_df) )
        test_pred_weights[column] = 0.0
    
def add_test_pred_data( prediction, columns, weight ):
    for column_idx, column in enumerate( columns ):
        test_pred_datas[column] += weight * prediction[:, column_idx]  
        test_pred_weights[column] += weight    

In [ ]:
validation_datas = {}
validation_counts = {}

def init_validation_data():
    for column in target_columns:
        validation_datas[column] = np.zeros( len(train_df) )
        validation_counts[column] = np.zeros( len(train_df) )
    
def add_validation_data( prediction, columns, idx ):
    for column_idx, column in enumerate( columns ):
        validation_datas[column][idx] += prediction[:, column_idx]  
        validation_counts[column][idx] += 1.0    

In [ ]:
init_test_pred_data()
init_validation_data()

In [ ]:
n_splits = 5

#cv = KFold(n_splits=n_splits, random_state=SEED)
#kfold_split = list( cv.split( x_train, y_train ) )

cv = GroupKFold( n_splits=n_splits )
kfold_split = list( cv.split( X=train_df.question_body, groups=train_df.question_body ) )   

In [ ]:
n_epochs = 50
patience = 5
scores   = []

batch_size = 32

num_features = x_train.shape[1]
num_labels   = y_train.shape[1]
    
for k, (train_idx, valid_idx) in enumerate( kfold_split ):
    
    print( "k:", k+1 )
    
    #train
    x_train_train = x_train[train_idx]
    y_train_train = y_train[train_idx]
    
    train_dataset = QuestDataset( x_train_train, y_train_train )
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True )
    
    #valid
    x_train_valid = x_train[valid_idx]
    y_train_valid = y_train[valid_idx]
    
    valid_dataset = QuestDataset( x_train_valid, y_train_valid )
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False )
    
    model = NeuralNet( num_features, num_labels )
    model.cuda()
    
    best_weights = copy.deepcopy( model.state_dict() ) 
    loss_fn = nn.BCEWithLogitsLoss()
    
    #optimizer = optim.Adam( model.parameters(), lr=2e-5 )
    optimizer = RAdam( model.parameters(), lr=2e-5 )
    
    schduler = CyclicScheduler2( min_lr=2e-6, max_lr=2e-5, period=20, warm_start=0, max_decay=0.9 )
    
    min_loss = np.inf
    counter = 0
    
    for epoch in range(n_epochs):
    #for epoch in tqdm( range(n_epochs) ):
        
        lr = schduler( epoch )
        adjust_learning_rate( optimizer, lr )
        #lr = get_learning_rate( optimizer )        
        
        model.train()
        train_loss = []
            
        for features, labels in train_loader:
            features = torch.tensor(features,dtype=torch.float32).cuda()
            labels   = torch.tensor(labels,dtype=torch.float32).cuda()   
            
            y_pred = model( features )
            loss = loss_fn( y_pred, labels )        
                           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss.append( loss.item() )
        
        avg_train_loss = np.mean( train_loss )
            
        model.eval()
        val_loss = []
        
        with torch.no_grad():
            for features, labels in valid_loader:
                features = torch.tensor(features,dtype=torch.float32).cuda()
                labels   = torch.tensor(labels,dtype=torch.float32).cuda()   
                
                y_pred = model( features )
                loss = loss_fn( y_pred, labels )
                val_loss.append( loss.item() )

        avg_val_loss = np.mean( val_loss )       
        
        if avg_val_loss < min_loss:
            min_loss = avg_val_loss
            counter = 0
            best_weights = copy.deepcopy( model.state_dict() ) 
                
        else:
            counter += 1
            # print('Early stopping: %i / %i' % (counter, self.patience))
            if counter >= patience and epoch > 12:
                # print('Early stopping at epoch', epoch + 1)
                break        
        
    model.load_state_dict( best_weights )    
    model.eval()
    valid_preds_fold = []
    valid_true_fold = []
    
    with torch.no_grad():
        for features, labels in valid_loader:
            features = torch.tensor(features,dtype=torch.float32).cuda()
            labels   = torch.tensor(labels,dtype=torch.float32).cuda()  
            y_pred = model( features )
            
            valid_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )
            valid_true_fold.extend( labels.cpu().data.numpy().tolist() )
       
    valid_preds_fold = np.array( valid_preds_fold )
    valid_true_fold = np.array( valid_true_fold )
    
    add_validation_data( valid_preds_fold, target_columns, valid_idx )
    score = mean_spearmanr_correlation_score( valid_preds_fold, valid_true_fold )
    print('Score:', score)
    
    scores.append(score)
    test_preds_fold  = []
    
    test_dataset = QuestDataset( x_test, None )
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False )

    with torch.no_grad():
        for features in test_loader:
            features = torch.tensor(features,dtype=torch.float32).cuda()
            y_pred = model( features )
            
            test_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

        test_preds_fold = np.array( test_preds_fold )
        add_test_pred_data( test_preds_fold, target_columns, 1.0 )
        
    del model, optimizer, loss_fn, best_weights
    del train_dataset, train_loader
    del valid_dataset, valid_loader  
    del test_dataset, test_loader
    torch.cuda.empty_cache()
    gc.collect() 

print( '================' )
print( 'Mean score:', np.mean(scores)) 

### validation

In [ ]:
validation0 = pd.DataFrame()

for column in target_columns:
    preds = validation_datas[column]
    count = validation_counts[column]
    count = np.where( count < 0.5, 1.0, count )
    
    validation0[column] = preds / count

In [ ]:
validation0.head()

In [ ]:
mean_spearmanr_correlation_score( train_df[target_columns].values, validation0.values )

### test

In [ ]:
test_preds0 = pd.read_csv( f"{folder}sample_submission.csv" )

for column in target_columns:
    output = test_pred_datas[column] / test_pred_weights[column]
    test_preds0[column] = output

In [ ]:
test_preds0.head()

# QUEST_005_07_DISTILBERT+LSTM (LSTM)

In [ ]:
LSTM_UNITS = 512
#LSTM_UNITS = 1024
DENSE_HIDDEN_UNITS = 6 * LSTM_UNITS

class MODEL_v001(nn.Module):
    def __init__(self, num_features, num_labels ):
        super().__init__()
        self.lstm1 = nn.LSTM( num_features , LSTM_UNITS, bidirectional=True, batch_first=True )
        self.lstm2 = nn.LSTM( LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True,dropout=0.2 )
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linearnorm = nn.LayerNorm(DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linearnorm2 = nn.LayerNorm(DENSE_HIDDEN_UNITS)
        self.linear_sub_out = nn.Linear(DENSE_HIDDEN_UNITS, num_labels)

    def forward(self, x, lengths=None):
        h_lstm1, _ = self.lstm1(x)
        h_lstm2, _ = self.lstm2(h_lstm1)

        avg_pool1 = torch.mean(h_lstm1, 1)
        avg_pool2 = torch.mean(h_lstm2, 1)
        max_pool2, _ = torch.max(h_lstm2, 1)

        h_conc = torch.cat((avg_pool1, max_pool2, avg_pool2), 1)
        h_conc_linear1 = self.linearnorm(F.relu(self.linear1(h_conc)))
        h_conc_linear2 = self.linearnorm2(F.relu(self.linear2(h_conc)))

        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        out = self.linear_sub_out(hidden)
        
        return out

In [ ]:
x_train = np.expand_dims( x_train, 1 )
x_test = np.expand_dims( x_test, 1 )

In [ ]:
init_test_pred_data()
init_validation_data()

In [ ]:
n_splits = 5

#cv = KFold(n_splits=n_splits, random_state=SEED)
#kfold_split = list( cv.split( x_train, y_train ) )

cv = GroupKFold( n_splits=n_splits )
kfold_split = list( cv.split( X=train_df.question_body, groups=train_df.question_body ) )   

In [ ]:
n_epochs = 8
patience = 4
scores   = []

batch_size = 8

num_features = x_train.shape[2]
num_labels   = y_train.shape[1]
    
for k, (train_idx, valid_idx) in enumerate( kfold_split ):
    
    print( "k:", k+1 )
    
    #train
    x_train_train = x_train[train_idx]
    y_train_train = y_train[train_idx]
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.from_numpy(x_train[train_idx]), 
        torch.from_numpy(y_train[train_idx])
    )
    
    train_loader = torch.utils.data.DataLoader( train_dataset, batch_size=batch_size, shuffle=True, drop_last=True )
    
    #valid
    valid_dataset = torch.utils.data.TensorDataset(
        torch.from_numpy(x_train[valid_idx]), 
        torch.from_numpy(y_train[valid_idx])
    )
    
    valid_loader = torch.utils.data.DataLoader( valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False )
    
    model = MODEL_v001( num_features, num_labels )
    model.cuda()
    
    best_weights = copy.deepcopy( model.state_dict() ) 
    loss_fn = nn.BCEWithLogitsLoss()
    
    optimizer = torch.optim.Adam( model.parameters() )
    #optimizer = optim.Adam( model.parameters(), lr=2e-5 )
    #optimizer = RAdam( model.parameters(), lr=2e-5 )
    
    min_loss = np.inf
    counter = 0
    
    for epoch in range(n_epochs):
    #for epoch in tqdm( range(n_epochs) ):   
        
        model.train()
        train_loss = []
            
        for features, labels in train_loader:
            features = torch.tensor(features,dtype=torch.float32).cuda()
            labels   = torch.tensor(labels,dtype=torch.float32).cuda()   
            
            y_pred = model( features )
            loss = loss_fn( y_pred, labels )        
                           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss.append( loss.item() )
        
        avg_train_loss = np.mean( train_loss )
            
        model.eval()
        val_loss = []
        
        with torch.no_grad():
            for features, labels in valid_loader:
                features = torch.tensor(features,dtype=torch.float32).cuda()
                labels   = torch.tensor(labels,dtype=torch.float32).cuda()   
                
                y_pred = model( features )
                loss = loss_fn( y_pred, labels )
                val_loss.append( loss.item() )

        avg_val_loss = np.mean( val_loss )       
        
        if avg_val_loss < min_loss:
            min_loss = avg_val_loss
            counter = 0
            best_weights = copy.deepcopy( model.state_dict() ) 
                
        else:
            counter += 1
            # print('Early stopping: %i / %i' % (counter, self.patience))
            if counter >= patience:
                # print('Early stopping at epoch', epoch + 1)
                break        
        
    model.load_state_dict( best_weights )    
    model.eval()
    valid_preds_fold = []
    valid_true_fold = []
    
    with torch.no_grad():
        for features, labels in valid_loader:
            features = torch.tensor(features,dtype=torch.float32).cuda()
            labels   = torch.tensor(labels,dtype=torch.float32).cuda()  
            y_pred = model( features )
            
            valid_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )
            valid_true_fold.extend( labels.cpu().data.numpy().tolist() )
       
    valid_preds_fold = np.array( valid_preds_fold )
    valid_true_fold = np.array( valid_true_fold )
    
    add_validation_data( valid_preds_fold, target_columns, valid_idx )
    score = mean_spearmanr_correlation_score( valid_preds_fold, valid_true_fold )
    print('Score:', score)
    
    scores.append(score)
    test_preds_fold  = []
    
    test_dataset = QuestDataset( x_test, None )
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False )

    with torch.no_grad():
        for features in test_loader:
            features = torch.tensor(features,dtype=torch.float32).cuda()
            y_pred = model( features )
            
            test_preds_fold.extend( torch.sigmoid( y_pred ).cpu().data.numpy().tolist() )

        test_preds_fold = np.array( test_preds_fold )
        add_test_pred_data( test_preds_fold, target_columns, 1.0 )
        
    del model, optimizer, loss_fn, best_weights
    del train_dataset, train_loader
    del valid_dataset, valid_loader  
    del test_dataset, test_loader
    torch.cuda.empty_cache()
    gc.collect() 

print( '================' )
print( 'Mean score:', np.mean(scores)) 

### validation

In [ ]:
validationLSTM = pd.DataFrame()

for column in target_columns:
    preds = validation_datas[column]
    count = validation_counts[column]
    count = np.where( count < 0.5, 1.0, count )
    
    validationLSTM[column] = preds / count

In [ ]:
validationLSTM.head()

In [ ]:
mean_spearmanr_correlation_score( validationLSTM.values, train_df[target_columns].values )

### submission

In [ ]:
test_predsLSTM = pd.read_csv( f"{folder}sample_submission.csv" )

for column in target_columns:
    output = test_pred_datas[column] / test_pred_weights[column]
    test_predsLSTM[column] = output

In [ ]:
test_predsLSTM.head()

In [ ]:
del train_df, test_df
del x_train, x_test, y_train
del validation_datas, validation_counts
del test_pred_datas, test_pred_weights

In [ ]:
blabla

# post processing

| Name                    | valid          | test_preds     |
|-------------------------|----------------|----------------|
| QUEST_005_07-distilbert | validation0    | test_preds0    |
| QUEST-003-55            | validationT    | test_predsT    |
| QUEST-006-55 -GPT2      | validationTG    | test_predsTG  |
| QUEST-005-04-1          | validationT2   | test_predsT2   |
| QUEST-005_07-lstm       | validationLSTM | test_predsLSTM |

In [ ]:
data_dir = '../input/google-quest-challenge/'
train = pd.read_csv(path_join(data_dir, 'train.csv'))
test = pd.read_csv(path_join(data_dir, 'test.csv'))
sample = pd.read_csv(path_join(data_dir, 'sample_submission.csv'))

print(train.shape, test.shape)
train.head()

In [ ]:
targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ]

In [ ]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

In [ ]:
from functools import partial
import scipy as sp

class OptimizedRounder(object):
    def __init__(self,correlation):
        self.correlation = correlation
        self.coef_ = 0
        self.score = 0

    def _kappa_loss(self, coef, X, y):
        a= X.copy()
        b=y.copy()
        X_p = pd.cut(a, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0,1,2])
        
        a[X_p == 0] = 0
        a[X_p == 2] = 1 

        #print("validation score = {}".format(spearmanr(a, b).correlation))
        if spearmanr(a, b).correlation < self.correlation:
            self.score = spearmanr(a, b).correlation
            return - spearmanr(a, b).correlation + (self.correlation - spearmanr(a, b).correlation + 1)**10
        else:
            self.score = spearmanr(a, b).correlation
            return - spearmanr(a, b).correlation

    def fit(self, X, y,coef_ini):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = coef_ini
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def coefficients(self):
        return self.coef_['x']
    def score_fc(self):
        return self.score

In [ ]:
from collections import Counter

def optimize_preds( y_df, x_df, preds_df ):
    
    for title in targets:
        
        y1 = np.asarray(y_df[title].copy())
        X1 = np.asarray(x_df[title].copy())
        
        x_original     = np.asarray(x_df[title].copy())
        preds_original = np.asarray(preds_df[title].copy())     

        correlation = spearmanr(y1, X1).correlation
        print(title)
        print(correlation)

        coefficients_target = [0,0]
        correlation = spearmanr(y1, X1).correlation
        okcor = spearmanr(y1, X1).correlation
        maxi = correlation
        liste = [[0.1,0.9],[0.2,0.9],[0.3,0.9],[0.3,0.8],[0.2,0.7],[0.3,0.7],[0.2,0.6],[0.5,0.9],[0.3,0.6],[0.4,0.8],[0.7,0.9],[0.8,0.9]]

        for L in liste:
            optR = OptimizedRounder(correlation)
            optR.fit(X1,y1,L)
            coefficients = optR.coefficients()
            if optR.score_fc() > maxi:
                maxi = optR.score_fc()
                coefficients_target = coefficients

        if maxi != spearmanr(y1, X1).correlation:
            oof = X1.copy()
            oof[oof > coefficients_target[1]] = 1
            oof[oof <= coefficients_target[0]] = 0

            X1 = np.asarray(preds_df[title].copy())
            oof_test = X1.copy()
            oof_test[oof_test > coefficients_target[1]] = 1
            oof_test[oof_test <= coefficients_target[0]] = 0

            score = spearmanr(y1, oof).correlation
            if score - okcor > 0.001:
                print("difference validation score = {}".format(score - okcor))
                x_df[title] = oof

                dist = Counter(x_df[title])

                if 0 in list(dist.keys()):
                    dist[0] /= len(x_df)
                if 1 in list(dist.keys()):
                    dist[1] /= len(x_df)

                acum = 0
                bound = {}

                if 0 in list(dist.keys()):
                    acum = dist[0]
                    bound[0] = np.percentile(preds_df[title], acum * 100)

                if 1 in list(dist.keys()):
                    acum = 1 - dist[1]
                    bound[1] = np.percentile(preds_df[title], acum * 100)

                def classify(x):
                    if 0 in list(dist.keys()):
                        if 1 in list(dist.keys()):
                            if x <= bound[0]:
                                return 0
                            elif x <= bound[1]:
                                return x
                            else:
                                return 1
                        else:
                            if x <= bound[0]:
                                return 0
                            else:
                                return x
                    else:
                        if 1 in list(dist.keys()):
                            if x <= bound[1]:
                                return x
                            else:
                                return 1
                        else:
                            return x

                final_pred = np.array(list(map(classify, preds_df[title])))
                if len(np.unique(oof_test)) != 1:
                    print(coefficients_target)
                    preds_df[title] = final_pred

In [ ]:

validationN=pd.read_csv('../input/nirjharbert003/validation_preds_df_003.csv') 
test_predsN=pd.read_csv('../input/nirjharbert003/submission_003.csv')

print("Nirjhar model : ")
compute_spearmanr_ignore_nan(np.asarray(train[targets].copy()), np.asarray(validationN[targets].copy()))


In [ ]:

validationN1=pd.read_csv('../input/dataset-nirjhar/validation_preds_df_002.csv') 
test_predsN1=pd.read_csv('../input/dataset-nirjhar/submission_002.csv')

print("Nirjhar model : ")
compute_spearmanr_ignore_nan(np.asarray(train[targets].copy()), np.asarray(validationN1[targets].copy()))


In [ ]:
data_table = []
data_table.append( {'Name': 'QUEST-003-55',            'valid': copy.deepcopy( validationT ),    'test_preds': copy.deepcopy( test_predsT ) } )
#data_table.append( {'Name': 'QUEST-005-04-1',          'valid': copy.deepcopy( validationT2 ),   'test_preds': copy.deepcopy( test_predsT2 ) } )
data_table.append( {'Name': 'QUEST-006-03-GPT2',        'valid': copy.deepcopy( validationTG ),   'test_preds': copy.deepcopy( test_predsTG ) } )
data_table.append( {'Name': 'QUEST-005-07-distilbert', 'valid': copy.deepcopy( validation0 ),    'test_preds': copy.deepcopy( test_preds0 ) } )
data_table.append( {'Name': 'QUEST-005-07-lstm',       'valid': copy.deepcopy( validationLSTM ), 'test_preds': copy.deepcopy( test_predsLSTM ) } )
data_table.append( {'Name': 'QUEST-Bert-Nirjhar',       'valid': copy.deepcopy( validationN ), 'test_preds': copy.deepcopy( test_predsN ) } )
data_table.append( {'Name': 'QUEST-Bert-Nirjhar1',       'valid': copy.deepcopy( validationN1 ), 'test_preds': copy.deepcopy( test_predsN1 ) } )

In [ ]:
n_original = len(data_table)
score_list = []

for i in range( n_original ):
    print( '=========================' )
    print( data_table[i]['Name'] )
    print( '=========================' )
    
    data_table[i]['valid'] = pd.DataFrame(data_table[i]['valid']).rank() / len(data_table[i]['valid'])
    data_table[i]['test_preds'] = pd.DataFrame(data_table[i]['test_preds']).rank() / len(data_table[i]['test_preds'])   
    
    data_table.append( copy.deepcopy( data_table[i] ) )
        
    score1 = compute_spearmanr_ignore_nan( np.asarray( train[targets].copy() ), np.asarray( data_table[i]['valid'].copy() ) )
    
    optimize_preds( train, data_table[i]['valid'], data_table[i]['test_preds'] )
    
    score2 = compute_spearmanr_ignore_nan( np.asarray( train[targets].copy() ), np.asarray( data_table[i]['valid'].copy() ) )
    score_list.append( [score1, score2] )

In [ ]:
for i in range( len(score_list) ):    
    score1, score2 = score_list[i][0], score_list[i][1]
    print( '{:.6f} -> {:.6f} ({:.6f}) : {}'.format( score1, score2, score2 - score1, data_table[i]['Name'] ) )

In [ ]:
validation_totale = train[targets].copy()
test_preds_totale = sample.copy()

In [ ]:
for title in targets:

    y1 = np.asarray(train[title].copy())

    X_list = [ np.asarray( x['valid'][title].copy() ) for x in data_table ]
    test_list = [ np.asarray( x['test_preds'][title].copy() ) for x in data_table ]     
    corr_list = [ spearmanr(y1, x).correlation for x in X_list ]

    i = np.argmax(corr_list)

    corr_best  = copy.deepcopy( corr_list[i] )
    x_best     = copy.deepcopy( X_list[i] )
    test_best  = copy.deepcopy( test_list[i] )
    
    for weight_range in np.arange( 0.45, 0.95, 0.005 ):
        
        corr_list2 = [c*c for c in corr_list]
        cmin = np.min( corr_list2 )
        cmax = np.max( corr_list2 )

        weights = ( corr_list2 - cmin ) / ( cmax - cmin ) 
        weights *= weight_range
        weights += ( 1.0 - weight_range )

        #print( weight_range, weights, title )

        x = np.average( X_list, axis=0, weights=weights )   
        x = rankdata( x ) / len( x )
        corr = spearmanr( y1, x ).correlation

        if corr > corr_best:
            corr_best  = copy.deepcopy( corr )
            x_best     = copy.deepcopy( x )

            t = np.average( test_list, axis=0, weights=weights )   
            t = rankdata( t ) / len( t )       
            test_best  = copy.deepcopy( t )  
            

    validation_totale[title] = copy.deepcopy( x_best )
    test_preds_totale[title] = copy.deepcopy( test_best )

In [ ]:
compute_spearmanr_ignore_nan(np.asarray(train[targets].copy()), np.asarray(validation_totale[targets].copy()))

In [ ]:
def function_spelling(row):
    if row in 'CULTURE':
        return 1
    else:
        return 0

validation_totale['question_type_spelling'] = train['category'].apply(function_spelling)*validation_totale['question_type_spelling']

def function_spelling2(row):
    if 'english.stackexchange' in str(row):
        return 1
    if 'ell.stackexchange' in str(row):
        return 1
    else:
        return 0
#validation_totale['question_type_spelling'] = train['question_user_page'].apply(function_spelling2)*validation_totale['question_type_spelling']

In [ ]:
compute_spearmanr_ignore_nan(np.asarray(train[targets].copy()), np.asarray(validation_totale[targets].copy()))

In [ ]:
submission = pd.read_csv(path_join(data_dir, 'sample_submission.csv'))

for title in targets:
    for i, row in test_preds_totale.iterrows():
        submission.loc[submission['qa_id'] == row['qa_id'], title] = row[title]
    

In [ ]:
submission['question_type_spelling'] = test['category'].apply(function_spelling)*submission['question_type_spelling']
#submission['question_type_spelling'] = test['question_user_page'].apply(function_spelling2)*validation['question_type_spelling']

In [ ]:
compute_spearmanr_ignore_nan(np.asarray(train[targets].copy()), np.asarray(validation_totale[targets].copy()))

In [ ]:
for title in targets:
    submission[title] = submission[title].apply(lambda x: 1 if x>1 else x)

In [ ]:
submission.to_csv("submission.csv", index = False)
submission.head()

In [ ]:
submission.describe()